In [2]:
import sys
sys.path.append('../')

In [3]:
import datetime
import math
from multiprocessing.pool import ThreadPool

import pandas
import numpy
import numpy.linalg
import scipy.linalg

from tqdm import tqdm

import lib.utils as utils
import lib.user_based_suggest as user_based_suggest

In [4]:
timus = pandas.read_csv('../data/timus.csv', index_col=0)

/home/ec2-user/mount_point/tfnewenv/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
timus.head()

,author_id,date,execution_time,judgement_result,language,memory_used,problem_id,submit_id,test_number
0,10004,2000-04-28 10:15:41,0.02,Accepted,Intel C++ 7,85,1000,184,0
1,10005,2000-05-12 13:13:38,0.00,Compilation error,Intel C++ 7,0,1000,185,0
2,10020,2000-09-22 15:58:11,0.01,Accepted,FreePascal 1.0.6,117,1000,220,0
3,10022,2000-09-26 17:17:34,0.01,Accepted,Intel C++ 7,81,1000,247,0
4,10033,2000-09-26 18:21:55,0.00,Compilation error,FreePascal 1.0.6,0,1000,276,0


In [6]:
filtered_timus = utils.filter_timus_by_unique_accepted(timus, lambda cnt: cnt > 400)

In [7]:
tc = utils.TestCase(filtered_timus, datetime.datetime.now() - datetime.timedelta(days=360))

In [8]:
sug = user_based_suggest.suggest(tc.get_train(), 5)

In [9]:
print 1

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(1)? (<ipython-input-9-c94594b6b28f>, line 1)

In [ ]:
s = utils.get_first_ac_by_author(tc.get_test())

In [ ]:
def get_score(suggest_by_author,
              next_accepted_by_author
             ):
    stoped_count = 0
    good_count = 0
    bad_count = 0
    for author in suggest_by_author:
        if author not in next_accepted_by_author:
            stoped_count += 1
            continue
        next_accepted = next_accepted_by_author[author]
        is_good = any([
            single_suggest.problem_index == next_accepted
            for single_suggest in suggest_by_author[author]
        ])
        if is_good:
            good_count += 1
        else:
            bad_count += 1
    print('good_count = {} bad_count = {} stoped_count = {}'.format(good_count, bad_count, stoped_count))
    return good_count / (good_count + bad_count)

In [ ]:
get_score(sug, s)

In [60]:
get_score(sug, s)

0.041666666666666664

In [19]:
s = sorted(s, key=lambda x: x.score, reverse=True)

In [21]:
for item in s:
    print(problems_encoder.decode(item.problem_index), item.score)

1589 0.1
1394 0.08333333333333333
1388 0.043478260869565216
1369 0.041666666666666664
1383 0.034482758620689655
1819 0.029411764705882353
1285 0.02824858757062147
1728 0.015217391304347827
2036 0.01
1075 0.009876543209876543
1726 0.008793969849246231
1727 0.005537974683544304
2028 0.005494505494505495
2032 0.005208333333333333
2029 0.004672897196261682
2026 0.004201680672268907
1725 0.004022988505747126
2030 0.003875968992248062
2027 0.0035087719298245615
2021 0.0031746031746031746
1731 0.002883031301482702
2034 0.0026041666666666665
2024 0.0025188916876574307
2019 0.0020408163265306124
1723 0.0019892014776925265
1496 0.0012598992080633548
1548 0.0
2016 0.0
2043 0.0
2041 0.0
2040 0.0
2039 0.0
2038 0.0
2042 0.0
2044 0.0
2048 0.0
2047 0.0
2051 0.0
2052 0.0
2053 0.0
2055 0.0
2060 0.0
2059 0.0
2058 0.0
2057 0.0
1799 0.0
2063 0.0
2065 0.0
2072 0.0
2064 0.0
2067 0.0
2062 0.0
2071 0.0
2076 0.0
1896 0.0
2089 0.0
2078 0.0
2081 0.0
2084 0.0
2080 0.0
2086 0.0
2079 0.0
2085 0.0
2083 0.0
2082 0.0
2

In [135]:
problems_encoder.encode(2006)

1000

In [108]:
U, S, VT = scipy.linalg.svd(solved_matrix, full_matrices=False)

In [109]:
SD = numpy.diag(S)

In [110]:
P = U.dot(SD).dot(VT)

In [111]:
numpy.linalg.norm(P - solved_matrix)

4.80038472230125e-12

In [119]:
P.shape

(18018, 1108)